**To find empty Car parking spaces**

First, let us mount our google drive. To do the same you need to authorize colab to access your drive.

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Let us go to our project location. I have created a folder project --> Car detection in my google drive

In [14]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/project/car detection"

car_detect.py	 logs	     parking.mp4  test_images
car_parking.mp4  output.mp4  rcnn.h5	  wait.py


Let us install mrcnn and twilio which are required for Car detection and Messaging.


In [3]:
!pip install -q mrcnn twilio

     |████████████████████████████████| 61kB 2.9MB/s 
     |████████████████████████████████| 993kB 10.1MB/s 


In [0]:
import os
import numpy as np
import cv2
import mrcnn.config
import mrcnn.utils
from mrcnn.model import MaskRCNN
from mrcnn import visualize
from pathlib import Path
from twilio.rest import Client

Configure the RCNN model by setting some config parameters.

In [0]:
class RCNNConfig(mrcnn.config.Config):
    NAME = "coco_pretrained_model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 1 + 80  # COCO dataset has 80 classes + one background class
    DETECTION_MIN_CONFIDENCE = 0.6




The get_car_boxes returns objects which are only Car, Bus or Truck.

Here is the list of
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [0]:
def get_car_boxes(boxes, class_ids):
    car_box = []

    for i, box in enumerate(boxes):
        if class_ids[i] in [3, 6, 8]:
            car_box.append(box)


    return np.array(car_box)

Setup your twilio account at https://www.twilio.com/ and configure it to send an SMS.

In [0]:
# Twilio config
twilio_account_sid = 'your twilio account sid'
twilio_auth_token = 'twilio auth token'
twilio_phone_number = 'FROM phone number'
destination_phone_number = 'TO phone number'
client = Client(twilio_account_sid, twilio_auth_token)

Load the model  and configure the input and output paths.
**VIDEO_SOURCE** is the path where the input is stored in drive. 
**VIDEO_STREAM_OUT** is the path where the output video will be saved


In [0]:
ROOT_DIR = Path("/content/drive/My Drive/project/car detection")

MODEL_DIR = os.path.join(ROOT_DIR, "logs")    

COCO_MODEL_PATH = os.path.join(ROOT_DIR, "rcnn.h5")

if not os.path.exists(COCO_MODEL_PATH):
    mrcnn.utils.download_trained_weights(COCO_MODEL_PATH)

IMAGE_DIR = os.path.join(ROOT_DIR, "images")    

VIDEO_SOURCE = "/content/drive/My Drive/project/car detection/car_parking.mp4"
VIDEO_STREAM_OUT = "/content/drive/My Drive/project/car detection/test_images/output.avi"


model = MaskRCNN(mode = "inference", model_dir = MODEL_DIR, config = RCNNConfig())

model.load_weights(COCO_MODEL_PATH, by_name = True)

The display_instances is the function which returns the image with the bounding boxes.


In [0]:
def display_instances(image, boxes):

    """
        take the image and results and apply the mask, box, and Label
    """
    n_instances = boxes.shape[0]

    colors = visualize.random_colors(n_instances)

    if not n_instances:

        print('NO INSTANCES TO DISPLAY')

    for i, color in enumerate(colors):

        if not np.any(boxes[i]):

            continue

        y1, x1, y2, x2 = boxes[i]

        #label = names[ids[i]]

        #score = scores[i] if scores is not None else None

        #caption = '{} {:.2f}'.format(label, score) if score else label

        #mask = masks[:, :, i]

        #image = visualize.apply_mask(image, mask, color)

        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

    return image


The below part reads the input video and sends a text message if it finds a parking space!!

In [21]:
# Location of parking spaces
parked_car_boxes = None

# How many frames of video we've seen in a row with a parking space open
free_space_frames = 0

# Have we sent an SMS alert yet?
sms_sent = False



video_capture = cv2.VideoCapture(VIDEO_SOURCE)
writer = None

while video_capture.isOpened():
    ret, frame = video_capture.read()

    if ret == False:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color
    rgb_image = frame[:, :, ::-1]        

    # Run the image through the Mask R-CNN model to get results.
    result = model.detect([rgb_image], verbose = 0)

    r = result[0]

    # The r variable will now have the results of detection:
    # - r['rois'] are the bounding box of each detected object
    # - r['class_ids'] are the class id (type) of each detected object
    # - r['scores'] are the confidence scores for each detection
    # - r['masks'] are the object masks for each detected object (which gives you the object outline)
    
    if parked_car_boxes is None:
      #This is the first frame
      parked_car_boxes = get_car_boxes(r['rois'], r['class_ids'])
    
    else:
      # we already know where the parking spaces are
      # 
      car_parking_spaces = get_car_boxes(r['rois'], r['class_ids'])
      
      overlaps = mrcnn.utils.compute_overlaps(parked_car_boxes, car_parking_spaces)
      
      # Assume no spaces are free until we find one that is free
      free_space = False
      
      for parking_area, overlap_areas in zip(parked_car_boxes, overlaps):
        # For this parking space, find the max amount it was covered by any
        # car that was detected in our image (doesn't really matter which car)
        max_IoU_overlap = np.max(overlap_areas)
        
        if max_IoU_overlap < 0.15:
          free_space = True
         
      if free_space:
          free_space_frames += 1
          print("free spaces frames = " , free_space_frames)
          
      else:
          free_space_frames = 0
          
      if free_space_frames > 2:
          print("Space available")
          font = cv2.FONT_HERSHEY_DUPLEX
          writer.write(cv2.putText(frame, f"SPACE AVAILABLE!", (10, 150), font, 3.0, (0, 255, 0), 2, cv2.FILLED))
          
          if not sms_sent:
            print("SENDING SMS...")
            message = client.messages.create(
                    body="Parking space open - go go go!!",
                    from_=twilio_phone_number,
                    to=destination_phone_number
                )
            sms_sent = True
            
      masked_frame = display_instances(frame, car_parking_spaces)

    
      if writer is None:
          fourcc = cv2.VideoWriter_fourcc(*"XVID")
          writer = cv2.VideoWriter(VIDEO_STREAM_OUT, fourcc, 30,(masked_frame.shape[1], masked_frame.shape[0]), True)

      writer.write(masked_frame)


    # Hit 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up everything when finished
print("[INFO] cleaning up...")
writer.release()

video_capture.release()


free spaces frames =  1
free spaces frames =  1
free spaces frames =  2
free spaces frames =  1
free spaces frames =  2
free spaces frames =  3
Space available
SENDING SMS...
free spaces frames =  1
[INFO] cleaning up...
